# Model Deployment - Part 1 (Practice Code)

In [1]:
!pip install -q -r ../module/requirements.txt
import os
os.chdir('../')
import shutil
import glob
import cv2
import numpy as np
import gradio as gr
import uvicorn
import typing
from fastapi import FastAPI, File
from pydantic import BaseModel

## Testing Components

In [19]:
# Components
img = gr.Image(shape=(1200, 600), type='numpy')
img_section = gr.AnnotatedImage()
box = gr.Textbox()

# function to predict(dummy)
def mask(img_):
    box = [0, 0, 600, 300]
    mask = np.concatenate((np.zeros((300, 600)), np.ones((300, 600))))/2
    return [(img_, [(box, "box")]), f'{img_.shape}']

# links the components with function
interface = gr.Interface(mask, img, [img_section, box])
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7871
Running on public URL: https://ae4f4b3c534dc73c58.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


In [3]:
# function to predict(dummy)
def mask(img_):
    box = [0,300, 600, 600]
    mask1 = np.concatenate((np.zeros((500, 1000)), np.ones((500, 1000))))/2
    mask2 = np.concatenate((np.ones((500, 1000)), np.zeros((500, 1000))))/2
    return (img_, [(box, "box"), (mask1, "mask1"), (mask2, "mask2")])

# Function on select trigger
def on_select(evt: gr.SelectData):
    return f'You selected {evt.value} at {evt.index} from {evt.target}'

# Function on clear button
def clear():
    return None, None, None

# Creates block of components and links it with the function
with gr.Blocks() as demo:
    gr.Markdown("# Select an image.")
    with gr.Row():
        with gr.Column():
            with gr.Box():
                with gr.Column():
                    inp = gr.Image(shape=(1000, 1000))
                    with gr.Row():
                        btn_clear = gr.Button('Clear')
                        btn_submit = gr.Button("Submit", variant='primary')
        out = gr.AnnotatedImage().style(height=512, width=512, color_map={'': ''})
    textbox = gr.Textbox()
    
    btn_submit.click(fn=mask, inputs=inp, outputs=out)
    btn_clear.click(clear, None, [inp, out, textbox])
    out.select(on_select, None, textbox)

demo.launch(share='True')

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://45e372f23efa49ea54.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


## Building a Demo Website

In [4]:
# Creating a demo directory containing all the files
demo_dir = 'demo'
if not os.path.exists(demo_dir):
    os.makedirs(demo_dir)
    
# Copying the scripts
for i in os.listdir('module'):
    if i.endswith('.txt') or i.endswith('.py'):
        if os.path.isfile(os.path.join('module', i)):
            shutil.copy2(os.path.join('module', i), demo_dir)
            print(f'[INFO] File "{os.path.join("module", i)}" has been copied to "{demo_dir}".')
            
# Copying some images for examples
example_dir = 'demo/examples'
if not os.path.exists(example_dir):
    os.makedirs(example_dir)
    
example_list = ['datasets/images/val/chicken_curry/1213873.jpg',
                'datasets/images/val/chocolate_cake/1520283.jpg',
                'datasets/images/val/hamburger/3508112.jpg',
                'datasets/images/val/pizza/2398925.jpg',
                'datasets/images/val/ramen/672622.jpg']

for i in example_list:
    if os.path.isfile(i):
        shutil.copy2(i, example_dir)
        print(f'[INFO] File "{i}" has been copied to "{example_dir}".')
        
# Copying the model file
model_path = 'models/yolov8_5class_10percent/best.onnx'
model_dirpath = 'demo/models'

if not os.path.exists(model_dirpath):
    os.makedirs(model_dirpath)

if os.path.isfile(model_path):
    shutil.copy2(model_path, model_dirpath)
    print(f'[INFO] File "{model_path}" has been copied to "{model_dirpath}".')

# Renaming the model file
if os.path.isfile(os.path.join(model_dirpath, 'best.onnx')):
    os.rename(os.path.join(model_dirpath, 'best.onnx'), os.path.join(model_dirpath, 'yolov8s-seg-v1.onnx'))
    print(f'[INFO] Model File is renamed to "{os.path.join(model_dirpath, "yolov8s-seg-v1.onnx")}".')

[INFO] File "module/requirements.txt" has been copied to "demo".
[INFO] File "module/predict.py" has been copied to "demo".
[INFO] File "module/engine.py" has been copied to "demo".
[INFO] File "module/utils.py" has been copied to "demo".
[INFO] File "datasets/images/val/chicken_curry/1213873.jpg" has been copied to "demo/examples".
[INFO] File "datasets/images/val/chocolate_cake/1520283.jpg" has been copied to "demo/examples".
[INFO] File "datasets/images/val/hamburger/3508112.jpg" has been copied to "demo/examples".
[INFO] File "datasets/images/val/pizza/2398925.jpg" has been copied to "demo/examples".
[INFO] File "datasets/images/val/ramen/672622.jpg" has been copied to "demo/examples".
[INFO] File "models/yolov8_5class_10percent/best.onnx" has been copied to "demo/models".
[INFO] Model File is renamed to "demo/models/yolov8s-seg-v1.onnx".


In [ ]:
os.chdir('demo')

In [11]:
from predict import predict

# Creating a predict function for the demo website
def gradio_predict(img):
    model_path='models/yolov8s-seg-v1.onnx'
    # Getting the prediction result for the image
    results = predict(image_array=img,
                      model_path=model_path,
                      conf_threshold=0.7)
    
    # formating the classes
    class_list = []
    for names in results['classes']:
        class_list.append(names.replace('_', ' '))
        
    # Validating the result
    if len(results['masks']) == 0:
        return (img, [([0, 0, 0, 0], 'No Food Detected')])
    else:
        # Isolating the result for every mask
        pred = []
        for i in range(len(results['masks'])):
            pred.append((results['masks'][i] / 2, class_list[results['class_ids'][i]]))
        return (img, pred)

# Creating a function when segment is selected
def on_annot_select(evt: gr.SelectData):
    return f'Selected Class: {evt.value}'

# Creating a function to clear all data
def on_clear_btn():
    return None, None, None

# Creating the UI
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    # Header
    gr.Markdown('<center><h1>Title</h1></center>')
    gr.Markdown('<center>Description</center>')
    
    # Body
    with gr.Row():
        # Image uploading
        with gr.Column(min_width=768):
            with gr.Box():
                with gr.Column():
                    input = gr.Image(type='numpy', 
                                     label='Image')
                    with gr.Row():
                        btn_clear = gr.Button(value='Clear')
                        btn_submit = gr.Button(value="Submit", 
                                               variant='primary')
                    gr.Examples(examples=glob.glob('examples/*.jpg'),
                                inputs=input)
        # Displaying resulted image
        output = gr.AnnotatedImage(label='Result').style(height=512, width=512, color_map={'': ''})
    
    # Additional info textbox 
    food_info_box = gr.Textbox(label='Food Info')
    
    # Footer
    gr.Markdown('article')
    
    # On selected event
    btn_submit.click(fn=gradio_predict, inputs=input, outputs=output)
    btn_clear.click(fn=on_clear_btn, inputs=None, outputs=[input, output, food_info_box])
    output.select(fn=on_annot_select, inputs=None, outputs=food_info_box)

# Launching the demo
demo.launch(share='True')

Running on local URL:  http://127.0.0.1:7864
Running on public URL: https://04db5891daf7b4cb20.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
